### Data Analysis (SQL)

In [23]:
import os
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

In [24]:
engine = create_engine(URL(
        #The account name is in the url to access the Snowflake UI, example: something.snowflakecomputing.com, where "something" is the account name.
        account="sn34773.europe-west2.gcp",
        user="AALFEREA91",
        password="",#I have deleted the real password
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="DW_AERO",
        schema="PUBLIC"
    ))

In [25]:
conn = engine.connect()
conn

Obtain the number of routes of each country as origin country in descending order. Show the name of the country

SELECT COUNT(cod_ruta) as numero_rutas,pais_aeropuerto FROM hc_rutas
INNER JOIN dm_aeropuertos ON cod_aeropuerto_ori=cod_aeropuerto
GROUP BY pais_aeropuerto
ORDER BY numero_rutas DESC

In [26]:
pd.read_sql_query('SELECT COUNT(cod_ruta) as numero_rutas,pais_aeropuerto\
                  FROM hc_rutas\
                  INNER JOIN dm_aeropuertos ON cod_aeropuerto_ori=cod_aeropuerto\
                  GROUP BY pais_aeropuerto\
                  ORDER BY numero_rutas DESC',
                  conn)

,numero_rutas,pais_aeropuerto
0,13193,United States
1,8197,China
2,2662,United Kingdom
3,2531,Spain
4,2353,Germany
...,...,...
222,1,Tuvalu
223,1,Falkland Islands (Malvinas)
224,1,Lesotho
225,1,Cocos (Keeling) Islands


Obtain the number of routes of each country as country of the airline in descending order. Show the name of the country

SELECT COUNT(cod_ruta) as numero_rutas,pais_aerolinea FROM hc_rutas as t1
INNER JOIN dm_aerolineas as t2 ON t1.cod_aerolinea=t2.cod_aerolinea
GROUP BY pais_aerolinea
ORDER BY numero_rutas desc

In [27]:
pd.read_sql_query('SELECT COUNT(cod_ruta) as numero_rutas,pais_aerolinea FROM hc_rutas as t1 \
                    INNER JOIN dm_aerolineas as t2 ON t1.cod_aerolinea=t2.cod_aerolinea \
                    GROUP BY pais_aerolinea \
                    ORDER BY numero_rutas desc',
                  conn)

,numero_rutas,pais_aerolinea
0,12785,United States
1,7262,China
2,3336,United Kingdom
3,2942,Germany
4,2760,Ireland
...,...,...
162,4,Republic of the Congo
163,4,Congo (Kinshasa)
164,4,Albania
165,2,Kyrgyzstan


Obtain the number of of routes of each equipment in descending order. Show the name of the equipments and only show those equipments with more than 2000 routes

SELECT numero_rutas,desc_equipamiento
FROM (SELECT count(cod_ruta) AS numero_rutas,cod_equipamiento
FROM co_rutas_equipamientos
GROUP BY cod_equipamiento) AS t1
INNER JOIN dm_equipamientos AS t2
ON t1.cod_equipamiento=t2.cod_equipamiento
WHERE numero_rutas>2000
ORDER BY numero_rutas DESC

In [28]:
pd.read_sql_query('SELECT numero_rutas,desc_equipamiento\
                    FROM (SELECT count(cod_ruta) AS numero_rutas,cod_equipamiento\
                    FROM co_rutas_equipamientos\
                    GROUP BY cod_equipamiento) AS t1\
                    INNER JOIN dm_equipamientos AS t2\
                    ON t1.cod_equipamiento=t2.cod_equipamiento\
                    WHERE numero_rutas>2000\
                    ORDER BY numero_rutas DESC',
                      conn)

,numero_rutas,desc_equipamiento
0,15184,Airbus A320-100/200
1,10183,Boeing 737-800 pax
2,7535,Airbus A319
3,3512,Airbus A321-100/200
4,3223,Boeing 737-800 (winglets) pax
5,2759,Boeing 737 all pax models
6,2446,Boeing 737-700 (winglets) pax
7,2180,Canadair Regional Jet


Obtain the number of routes of each equipment for the routes whose origin airport is Barajas (1229) in descending order

SELECT count(t3.cod_ruta) AS numero_rutas,desc_equipamiento
FROM 
    (SELECT t1.cod_ruta,cod_equipamiento
    FROM
        (SELECT cod_ruta,cod_aeropuerto_ori
        FROM hc_rutas
        WHERE cod_aeropuerto_ori='1229') AS t1
    INNER JOIN co_rutas_equipamientos AS t2
    ON t1.cod_ruta=t2.cod_ruta) AS t3
 INNER JOIN dm_equipamientos AS t4
 ON t3.cod_equipamiento=t4.cod_equipamiento
 GROUP BY desc_equipamiento
 ORDER BY numero_rutas DESC

In [29]:
pd.read_sql_query("SELECT COUNT(t3.cod_ruta) AS numero_rutas,desc_equipamiento\
                    FROM\
                        (SELECT t1.cod_ruta,cod_equipamiento\
                        FROM\
                            (SELECT cod_ruta,cod_aeropuerto_ori\
                            FROM hc_rutas\
                            WHERE cod_aeropuerto_ori='1229') AS t1\
                        INNER JOIN co_rutas_equipamientos AS t2\
                        ON t1.cod_ruta=t2.cod_ruta) AS t3\
                     INNER JOIN dm_equipamientos AS t4\
                     ON t3.cod_equipamiento=t4.cod_equipamiento\
                     GROUP BY desc_equipamiento\
                     ORDER BY numero_rutas DESC",
                      conn)

,numero_rutas,desc_equipamiento
0,62,Boeing 737-800 pax
1,61,Airbus A320-100/200
2,55,Airbus A319
3,29,Airbus A321-100/200
4,27,Airbus A330-300
5,20,Airbus A318/319/320/321
6,20,Airbus A340-600
7,19,Embraer 190
8,18,Airbus A330-200
9,18,Canadair Regional Jet 1000


Show the name of the equipment. -Obtain the two countries in which there are more routes

SELECT *
FROM
  (SELECT count(t3.cod_ruta) AS total_rutas,t3.pais_or AS pais_or,t4.cod_pais AS pais_des
    FROM (SELECT cod_ruta,cod_aeropuerto_ori,cod_aeropuerto_des,t2.cod_pais AS pais_or
    FROM hc_rutas AS t1
    INNER JOIN dm_aeropuertos AS t2
    ON t1.cod_aeropuerto_ori=t2.cod_aeropuerto) AS t3
    INNER JOIN dm_aeropuertos AS t4
    ON t3.cod_aeropuerto_des=t4.cod_aeropuerto
    GROUP BY pais_or,pais_des)
ORDER BY total_rutas DESC

In [30]:
pd.read_sql_query("SELECT *\
                    FROM\
                      (SELECT count(t3.cod_ruta) AS total_rutas,t3.pais_or AS pais_or,t4.cod_pais AS pais_des\
                        FROM (SELECT cod_ruta,cod_aeropuerto_ori,cod_aeropuerto_des,t2.cod_pais AS pais_or\
                        FROM hc_rutas AS t1\
                        INNER JOIN dm_aeropuertos AS t2\
                        ON t1.cod_aeropuerto_ori=t2.cod_aeropuerto) AS t3\
                        INNER JOIN dm_aeropuertos AS t4\
                        ON t3.cod_aeropuerto_des=t4.cod_aeropuerto\
                        GROUP BY pais_or,pais_des)\
                    ORDER BY total_rutas DESC",
                      conn).head(20)
#Observamos que los paises entre los que mas rutas hay son GBR-ESP ESP-GBR

,total_rutas,pais_or,pais_des
0,10673,USA,USA
1,6946,CHN,CHN
2,1188,BRA,BRA
3,1149,CAN,CAN
4,1057,IND,IND
5,940,RUS,RUS
6,772,AUS,AUS
7,631,JPN,JPN
8,607,IDN,IDN
9,579,ESP,ESP
